# Data Generation using Modeling and Simulation for Machine Learning
## Pendulum Physics Simulator

**Name:** Aindri Singh  
**Roll Number:** 102316039  
**Course:** UCS654  
**Assignment:** 6  
**Date:** February 24, 2026

---

## Overview
This notebook demonstrates data generation using a **Pendulum Physics Simulator** to create training data for machine learning models. The simulator models the dynamics of a damped pendulum system and generates 1000 simulations with varying parameters.

## Step 1: Install Required Libraries

Installing all necessary packages for simulation, data processing, visualization, and machine learning.

In [ ]:
# Install required packages
!pip install numpy scipy pandas matplotlib seaborn scikit-learn xgboost lightgbm catboost plotly tqdm

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.integrate import odeint
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Machine Learning Libraries
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# Set random seed for reproducibility
np.random.seed(42)

# Plotting settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ All libraries imported successfully!")

## Step 2: Pendulum Simulator Implementation

### Physics Background
A damped pendulum is governed by the differential equation:

$$\frac{d^2\theta}{dt^2} + \frac{b}{m}\frac{d\theta}{dt} + \frac{g}{L}\sin(\theta) = 0$$

Where:
- θ = angular displacement
- L = length of pendulum (m)
- m = mass of bob (kg)
- b = damping coefficient (kg/s)
- g = gravitational acceleration (9.81 m/s²)

The simulator will track various physical properties over time.

In [ ]:
class PendulumSimulator:
    """
    A physics-based pendulum simulator that models damped pendulum motion.
    
    This simulator can be used to generate synthetic data for machine learning
    by varying input parameters and recording output metrics.
    """
    
    def __init__(self, length, mass, damping, initial_angle, initial_velocity, gravity=9.81):
        """
        Initialize the pendulum simulator with physical parameters.
        
        Parameters:
        -----------
        length : float
            Length of the pendulum (meters)
        mass : float
            Mass of the bob (kilograms)
        damping : float
            Damping coefficient (kg/s)
        initial_angle : float
            Initial angular displacement (radians)
        initial_velocity : float
            Initial angular velocity (rad/s)
        gravity : float
            Gravitational acceleration (m/s²), default is 9.81
        """
        self.length = length
        self.mass = mass
        self.damping = damping
        self.initial_angle = initial_angle
        self.initial_velocity = initial_velocity
        self.gravity = gravity
        
        # Simulation results
        self.time = None
        self.theta = None
        self.omega = None
        
    def _derivative(self, state, t):
        """
        Compute the derivatives for the pendulum ODE.
        
        state[0] = theta (angle)
        state[1] = omega (angular velocity)
        """
        theta, omega = state
        dtheta_dt = omega
        domega_dt = -(self.damping / self.mass) * omega - (self.gravity / self.length) * np.sin(theta)
        return [dtheta_dt, domega_dt]
    
    def simulate(self, duration=10.0, dt=0.01):
        """
        Run the pendulum simulation.
        
        Parameters:
        -----------
        duration : float
            Simulation duration (seconds)
        dt : float
            Time step (seconds)
        
        Returns:
        --------
        dict : Dictionary containing simulation metrics
        """
        # Time array
        self.time = np.arange(0, duration, dt)
        
        # Initial state
        initial_state = [self.initial_angle, self.initial_velocity]
        
        # Solve ODE
        solution = odeint(self._derivative, initial_state, self.time)
        self.theta = solution[:, 0]
        self.omega = solution[:, 1]
        
        # Calculate output metrics
        metrics = self._calculate_metrics()
        
        return metrics
    
    def _calculate_metrics(self):
        """
        Calculate various output metrics from the simulation.
        """
        # Maximum angle reached
        max_angle = np.max(np.abs(self.theta))
        
        # Final angle (steady state)
        final_angle = np.abs(self.theta[-1])
        
        # Maximum angular velocity
        max_velocity = np.max(np.abs(self.omega))
        
        # Time to settle (when angle < 5% of initial angle)
        threshold = 0.05 * np.abs(self.initial_angle)
        settle_mask = np.abs(self.theta) < threshold
        if np.any(settle_mask):
            settle_time = self.time[np.argmax(settle_mask)]
        else:
            settle_time = self.time[-1]
        
        # Total energy dissipated
        initial_energy = 0.5 * self.mass * (self.length * self.initial_velocity)**2 + \
                        self.mass * self.gravity * self.length * (1 - np.cos(self.initial_angle))
        final_energy = 0.5 * self.mass * (self.length * self.omega[-1])**2 + \
                      self.mass * self.gravity * self.length * (1 - np.cos(self.theta[-1]))
        energy_dissipated = initial_energy - final_energy
        
        # Number of oscillations (zero crossings)
        zero_crossings = np.sum(np.diff(np.sign(self.theta)) != 0) / 2
        
        # Average period (if oscillating)
        if zero_crossings > 0:
            period = settle_time / zero_crossings if zero_crossings > 0 else 0
        else:
            period = 0
        
        return {
            'max_angle': max_angle,
            'final_angle': final_angle,
            'max_velocity': max_velocity,
            'settle_time': settle_time,
            'energy_dissipated': energy_dissipated,
            'oscillation_count': zero_crossings,
            'period': period
        }
    
    def plot(self):
        """
        Visualize the pendulum simulation results.
        """
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        
        # Angle over time
        axes[0, 0].plot(self.time, np.degrees(self.theta), 'b-', linewidth=2)
        axes[0, 0].set_xlabel('Time (s)', fontsize=12)
        axes[0, 0].set_ylabel('Angle (degrees)', fontsize=12)
        axes[0, 0].set_title('Angular Displacement vs Time', fontsize=14, fontweight='bold')
        axes[0, 0].grid(True, alpha=0.3)
        
        # Angular velocity over time
        axes[0, 1].plot(self.time, self.omega, 'r-', linewidth=2)
        axes[0, 1].set_xlabel('Time (s)', fontsize=12)
        axes[0, 1].set_ylabel('Angular Velocity (rad/s)', fontsize=12)
        axes[0, 1].set_title('Angular Velocity vs Time', fontsize=14, fontweight='bold')
        axes[0, 1].grid(True, alpha=0.3)
        
        # Phase space
        axes[1, 0].plot(np.degrees(self.theta), self.omega, 'g-', linewidth=2, alpha=0.7)
        axes[1, 0].scatter(np.degrees(self.theta[0]), self.omega[0], c='blue', s=100, zorder=5, label='Start')
        axes[1, 0].scatter(np.degrees(self.theta[-1]), self.omega[-1], c='red', s=100, zorder=5, label='End')
        axes[1, 0].set_xlabel('Angle (degrees)', fontsize=12)
        axes[1, 0].set_ylabel('Angular Velocity (rad/s)', fontsize=12)
        axes[1, 0].set_title('Phase Space Plot', fontsize=14, fontweight='bold')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
        
        # Energy over time
        kinetic_energy = 0.5 * self.mass * (self.length * self.omega)**2
        potential_energy = self.mass * self.gravity * self.length * (1 - np.cos(self.theta))
        total_energy = kinetic_energy + potential_energy
        
        axes[1, 1].plot(self.time, kinetic_energy, 'b-', label='Kinetic Energy', linewidth=2)
        axes[1, 1].plot(self.time, potential_energy, 'r-', label='Potential Energy', linewidth=2)
        axes[1, 1].plot(self.time, total_energy, 'g--', label='Total Energy', linewidth=2)
        axes[1, 1].set_xlabel('Time (s)', fontsize=12)
        axes[1, 1].set_ylabel('Energy (Joules)', fontsize=12)
        axes[1, 1].set_title('Energy vs Time', fontsize=14, fontweight='bold')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

print("✓ PendulumSimulator class defined successfully!")

## Step 3: Define Parameter Bounds

### Simulation Parameters and Their Bounds:

| Parameter | Symbol | Lower Bound | Upper Bound | Unit | Description |
|-----------|--------|-------------|-------------|------|-------------|
| Length | L | 0.5 | 3.0 | m | Length of pendulum rod |
| Mass | m | 0.1 | 2.0 | kg | Mass of the bob |
| Damping | b | 0.0 | 0.5 | kg/s | Damping coefficient (air resistance) |
| Initial Angle | θ₀ | 0.1 | 3.0 | rad | Starting angular position (0.1 to ~170°) |
| Initial Velocity | ω₀ | -2.0 | 2.0 | rad/s | Starting angular velocity |

These ranges are chosen to represent realistic physical scenarios while ensuring interesting dynamics.

In [ ]:
# Define parameter bounds
PARAMETER_BOUNDS = {
    'length': (0.5, 3.0),          # meters
    'mass': (0.1, 2.0),            # kilograms
    'damping': (0.0, 0.5),         # kg/s
    'initial_angle': (0.1, 3.0),   # radians (≈5.7° to 172°)
    'initial_velocity': (-2.0, 2.0) # rad/s
}

# Display bounds table
bounds_df = pd.DataFrame([
    {'Parameter': 'Length (L)', 'Lower': 0.5, 'Upper': 3.0, 'Unit': 'm'},
    {'Parameter': 'Mass (m)', 'Lower': 0.1, 'Upper': 2.0, 'Unit': 'kg'},
    {'Parameter': 'Damping (b)', 'Lower': 0.0, 'Upper': 0.5, 'Unit': 'kg/s'},
    {'Parameter': 'Initial Angle (θ₀)', 'Lower': 0.1, 'Upper': 3.0, 'Unit': 'rad'},
    {'Parameter': 'Initial Velocity (ω₀)', 'Lower': -2.0, 'Upper': 2.0, 'Unit': 'rad/s'}
])

print("\n📊 PARAMETER BOUNDS")
print("=" * 60)
print(bounds_df.to_string(index=False))
print("=" * 60)

## Step 4: Single Simulation Example

Let's run a single simulation to verify the simulator works correctly.

In [ ]:
# Create a sample pendulum
sample_pendulum = PendulumSimulator(
    length=1.5,
    mass=0.5,
    damping=0.15,
    initial_angle=np.pi/3,  # 60 degrees
    initial_velocity=0.0
)

# Run simulation
metrics = sample_pendulum.simulate(duration=10.0, dt=0.01)

# Display results
print("\n🔬 SAMPLE SIMULATION RESULTS")
print("=" * 60)
print(f"Input Parameters:")
print(f"  Length: {sample_pendulum.length:.2f} m")
print(f"  Mass: {sample_pendulum.mass:.2f} kg")
print(f"  Damping: {sample_pendulum.damping:.3f} kg/s")
print(f"  Initial Angle: {np.degrees(sample_pendulum.initial_angle):.2f}°")
print(f"  Initial Velocity: {sample_pendulum.initial_velocity:.2f} rad/s\n")

print(f"Output Metrics:")
for key, value in metrics.items():
    print(f"  {key.replace('_', ' ').title()}: {value:.4f}")
print("=" * 60)

# Visualize
sample_pendulum.plot()

## Step 5: Generate 1000 Simulations

Now we'll generate 1000 simulations with randomly sampled parameters within the defined bounds.

In [ ]:
def generate_random_parameters(bounds, n_samples=1):
    """
    Generate random parameters within specified bounds.
    
    Parameters:
    -----------
    bounds : dict
        Dictionary with parameter names as keys and (min, max) tuples as values
    n_samples : int
        Number of parameter sets to generate
    
    Returns:
    --------
    list : List of dictionaries containing random parameter values
    """
    samples = []
    for _ in range(n_samples):
        sample = {}
        for param, (lower, upper) in bounds.items():
            sample[param] = np.random.uniform(lower, upper)
        samples.append(sample)
    return samples

def run_simulations(n_simulations=1000, duration=10.0):
    """
    Generate multiple simulations with random parameters.
    
    Parameters:
    -----------
    n_simulations : int
        Number of simulations to run
    duration : float
        Duration of each simulation (seconds)
    
    Returns:
    --------
    pd.DataFrame : DataFrame containing all simulation data
    """
    print(f"\n🚀 Generating {n_simulations} simulations...\n")
    
    # Generate random parameters
    param_samples = generate_random_parameters(PARAMETER_BOUNDS, n_simulations)
    
    # Store results
    all_results = []
    
    # Run simulations with progress bar
    for i, params in enumerate(tqdm(param_samples, desc="Running simulations")):
        try:
            # Create pendulum
            pendulum = PendulumSimulator(
                length=params['length'],
                mass=params['mass'],
                damping=params['damping'],
                initial_angle=params['initial_angle'],
                initial_velocity=params['initial_velocity']
            )
            
            # Run simulation
            metrics = pendulum.simulate(duration=duration)
            
            # Combine input parameters and output metrics
            result = {
                'simulation_id': i + 1,
                **params,
                **metrics
            }
            
            all_results.append(result)
            
        except Exception as e:
            print(f"\nWarning: Simulation {i+1} failed: {str(e)}")
            continue
    
    # Convert to DataFrame
    df = pd.DataFrame(all_results)
    
    print(f"\n✓ Successfully generated {len(df)} simulations!")
    print(f"✓ Dataset shape: {df.shape}")
    
    return df

# Generate the dataset
simulation_data = run_simulations(n_simulations=1000, duration=10.0)

# Display first few rows
print("\n📋 First 5 simulations:")
print(simulation_data.head())

## Data Exploration and Visualization

In [ ]:
# Statistical summary
print("\n📊 STATISTICAL SUMMARY")
print("=" * 80)
print(simulation_data.describe())
print("=" * 80)

# Check for missing values
print(f"\n🔍 Missing values: {simulation_data.isnull().sum().sum()}")
print(f"✓ Dataset info:")
print(simulation_data.info())

In [ ]:
# Visualize parameter distributions
fig, axes = plt.subplots(3, 2, figsize=(15, 12))
fig.suptitle('Input Parameter Distributions', fontsize=16, fontweight='bold', y=1.00)

input_params = ['length', 'mass', 'damping', 'initial_angle', 'initial_velocity']

for idx, param in enumerate(input_params):
    row = idx // 2
    col = idx % 2
    axes[row, col].hist(simulation_data[param], bins=40, color='skyblue', edgecolor='black', alpha=0.7)
    axes[row, col].set_xlabel(param.replace('_', ' ').title(), fontsize=12)
    axes[row, col].set_ylabel('Frequency', fontsize=12)
    axes[row, col].set_title(f'{param.replace("_", " ").title()} Distribution', fontsize=13, fontweight='bold')
    axes[row, col].grid(True, alpha=0.3)

# Hide the extra subplot
axes[2, 1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Visualize output metrics distributions
fig, axes = plt.subplots(3, 3, figsize=(16, 12))
fig.suptitle('Output Metrics Distributions', fontsize=16, fontweight='bold', y=1.00)

output_metrics = ['max_angle', 'final_angle', 'max_velocity', 'settle_time', 
                 'energy_dissipated', 'oscillation_count', 'period']

for idx, metric in enumerate(output_metrics):
    row = idx // 3
    col = idx % 3
    axes[row, col].hist(simulation_data[metric], bins=40, color='coral', edgecolor='black', alpha=0.7)
    axes[row, col].set_xlabel(metric.replace('_', ' ').title(), fontsize=11)
    axes[row, col].set_ylabel('Frequency', fontsize=11)
    axes[row, col].set_title(f'{metric.replace("_", " ").title()}', fontsize=12, fontweight='bold')
    axes[row, col].grid(True, alpha=0.3)

# Hide extra subplots
axes[2, 1].axis('off')
axes[2, 2].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Correlation heatmap
plt.figure(figsize=(14, 10))
correlation_matrix = simulation_data.drop('simulation_id', axis=1).corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix: Input Parameters vs Output Metrics', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Print strong correlations
print("\n🔗 Strong Correlations (|r| > 0.5):")
print("=" * 60)
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_value = correlation_matrix.iloc[i, j]
        if abs(corr_value) > 0.5:
            print(f"{correlation_matrix.columns[i]:25s} ↔ {correlation_matrix.columns[j]:25s}: {corr_value:7.3f}")
print("=" * 60)

## Save Dataset

In [ ]:
# Save to CSV
simulation_data.to_csv('pendulum_simulations.csv', index=False)
print("✓ Dataset saved as 'pendulum_simulations.csv'")

# Display dataset info
print(f"\n📁 Dataset Information:")
print(f"  Total simulations: {len(simulation_data)}")
print(f"  Input features: {len(input_params)}")
print(f"  Output metrics: {len(output_metrics)}")
print(f"  Total columns: {len(simulation_data.columns)}")

## Step 6: Machine Learning Model Comparison

We'll train multiple ML models to predict the **settle_time** (how long it takes for the pendulum to come to rest) based on the input parameters.

### Models to Compare:
1. Linear Regression
2. Ridge Regression
3. Lasso Regression
4. ElasticNet
5. Decision Tree
6. Random Forest
7. Gradient Boosting
8. XGBoost
9. LightGBM
10. CatBoost

In [ ]:
# Prepare data for ML
print("\n🎯 Preparing data for Machine Learning...\n")

# Define features (input parameters) and target (we'll predict settle_time)
feature_columns = ['length', 'mass', 'damping', 'initial_angle', 'initial_velocity']
target_column = 'settle_time'

X = simulation_data[feature_columns]
y = simulation_data[target_column]

# Split data into train and test sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"✓ Training set size: {X_train.shape[0]} samples")
print(f"✓ Test set size: {X_test.shape[0]} samples")
print(f"✓ Number of features: {X_train.shape[1]}")
print(f"✓ Target variable: {target_column}")

In [ ]:
# Define models to compare
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0, random_state=42),
    'Lasso Regression': Lasso(alpha=0.01, random_state=42, max_iter=5000),
    'ElasticNet': ElasticNet(alpha=0.01, l1_ratio=0.5, random_state=42, max_iter=5000),
    'Decision Tree': DecisionTreeRegressor(max_depth=10, random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=15, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42, n_jobs=-1),
    'LightGBM': LGBMRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42, n_jobs=-1, verbose=-1),
    'CatBoost': CatBoostRegressor(iterations=100, depth=5, learning_rate=0.1, random_state=42, verbose=0)
}

print(f"\n🤖 Training {len(models)} Machine Learning Models...\n")
print("Models to compare:")
for i, model_name in enumerate(models.keys(), 1):
    print(f"  {i}. {model_name}")

In [ ]:
# Train and evaluate models
import time

results = []

for model_name, model in tqdm(models.items(), desc="Training models"):
    try:
        # Start timer
        start_time = time.time()
        
        # Train model
        model.fit(X_train_scaled, y_train)
        
        # Training time
        training_time = time.time() - start_time
        
        # Make predictions
        y_train_pred = model.predict(X_train_scaled)
        y_test_pred = model.predict(X_test_scaled)
        
        # Calculate metrics
        train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
        test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
        train_mae = mean_absolute_error(y_train, y_train_pred)
        test_mae = mean_absolute_error(y_test, y_test_pred)
        train_r2 = r2_score(y_train, y_train_pred)
        test_r2 = r2_score(y_test, y_test_pred)
        
        # Cross-validation score (5-fold)
        cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, 
                                   scoring='neg_mean_squared_error', n_jobs=-1)
        cv_rmse = np.sqrt(-cv_scores.mean())
        
        # Store results
        results.append({
            'Model': model_name,
            'Train RMSE': train_rmse,
            'Test RMSE': test_rmse,
            'Train MAE': train_mae,
            'Test MAE': test_mae,
            'Train R²': train_r2,
            'Test R²': test_r2,
            'CV RMSE': cv_rmse,
            'Training Time (s)': training_time
        })
        
    except Exception as e:
        print(f"\nError training {model_name}: {str(e)}")
        continue

# Create results DataFrame
results_df = pd.DataFrame(results)

# Sort by Test R² score (descending)
results_df = results_df.sort_values('Test R²', ascending=False).reset_index(drop=True)

print("\n" + "=" * 100)
print("🏆 MODEL COMPARISON RESULTS - SORTED BY TEST R² SCORE")
print("=" * 100)
print(results_df.to_string(index=False))
print("=" * 100)

In [ ]:
# Identify best model
best_model_idx = results_df['Test R²'].idxmax()
best_model_name = results_df.loc[best_model_idx, 'Model']
best_model_r2 = results_df.loc[best_model_idx, 'Test R²']
best_model_rmse = results_df.loc[best_model_idx, 'Test RMSE']

print(f"\n🥇 BEST MODEL: {best_model_name}")
print("=" * 60)
print(f"  Test R² Score: {best_model_r2:.6f}")
print(f"  Test RMSE: {best_model_rmse:.6f}")
print(f"  Test MAE: {results_df.loc[best_model_idx, 'Test MAE']:.6f}")
print(f"  CV RMSE: {results_df.loc[best_model_idx, 'CV RMSE']:.6f}")
print(f"  Training Time: {results_df.loc[best_model_idx, 'Training Time (s)']:.4f} seconds")
print("=" * 60)

# Save results
results_df.to_csv('model_comparison_results.csv', index=False)
print("\n✓ Results saved as 'model_comparison_results.csv'")

## Visualization of Model Performance

In [ ]:
# Plot model comparison - Test R² Score
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Test R² Score
axes[0, 0].barh(results_df['Model'], results_df['Test R²'], color='steelblue', edgecolor='black')
axes[0, 0].set_xlabel('Test R² Score', fontsize=12, fontweight='bold')
axes[0, 0].set_title('Test R² Score Comparison', fontsize=14, fontweight='bold')
axes[0, 0].axvline(x=results_df['Test R²'].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3, axis='x')

# Test RMSE
axes[0, 1].barh(results_df['Model'], results_df['Test RMSE'], color='coral', edgecolor='black')
axes[0, 1].set_xlabel('Test RMSE', fontsize=12, fontweight='bold')
axes[0, 1].set_title('Test RMSE Comparison (Lower is Better)', fontsize=14, fontweight='bold')
axes[0, 1].axvline(x=results_df['Test RMSE'].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3, axis='x')

# Test MAE
axes[1, 0].barh(results_df['Model'], results_df['Test MAE'], color='lightgreen', edgecolor='black')
axes[1, 0].set_xlabel('Test MAE', fontsize=12, fontweight='bold')
axes[1, 0].set_title('Test MAE Comparison (Lower is Better)', fontsize=14, fontweight='bold')
axes[1, 0].axvline(x=results_df['Test MAE'].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3, axis='x')

# Training Time
axes[1, 1].barh(results_df['Model'], results_df['Training Time (s)'], color='plum', edgecolor='black')
axes[1, 1].set_xlabel('Training Time (seconds)', fontsize=12, fontweight='bold')
axes[1, 1].set_title('Training Time Comparison (Lower is Better)', fontsize=14, fontweight='bold')
axes[1, 1].axvline(x=results_df['Training Time (s)'].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('model_comparison_charts.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Comparison charts saved as 'model_comparison_charts.png'")

## Best Model - Detailed Analysis

In [ ]:
# Train the best model
best_model = models[best_model_name]
best_model.fit(X_train_scaled, y_train)

# Predictions
y_train_pred = best_model.predict(X_train_scaled)
y_test_pred = best_model.predict(X_test_scaled)

# Prediction vs Actual plots
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Training set
axes[0].scatter(y_train, y_train_pred, alpha=0.5, s=30, color='blue', edgecolors='black', linewidth=0.5)
axes[0].plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'r--', lw=3, label='Perfect Prediction')
axes[0].set_xlabel('Actual Settle Time (s)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Predicted Settle Time (s)', fontsize=12, fontweight='bold')
axes[0].set_title(f'{best_model_name} - Training Set\nR² = {r2_score(y_train, y_train_pred):.4f}', 
                 fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Test set
axes[1].scatter(y_test, y_test_pred, alpha=0.5, s=30, color='green', edgecolors='black', linewidth=0.5)
axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=3, label='Perfect Prediction')
axes[1].set_xlabel('Actual Settle Time (s)', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Predicted Settle Time (s)', fontsize=12, fontweight='bold')
axes[1].set_title(f'{best_model_name} - Test Set\nR² = {r2_score(y_test, y_test_pred):.4f}', 
                 fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('best_model_predictions.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Best model analysis saved as 'best_model_predictions.png'")

In [ ]:
# Feature importance (for tree-based models)
if hasattr(best_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'Feature': feature_columns,
        'Importance': best_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    print("\n📊 FEATURE IMPORTANCE")
    print("=" * 60)
    print(feature_importance.to_string(index=False))
    print("=" * 60)
    
    # Plot feature importance
    plt.figure(figsize=(10, 6))
    plt.barh(feature_importance['Feature'], feature_importance['Importance'], 
             color='teal', edgecolor='black', linewidth=1.5)
    plt.xlabel('Importance Score', fontsize=12, fontweight='bold')
    plt.title(f'Feature Importance - {best_model_name}', fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3, axis='x')
    plt.tight_layout()
    plt.savefig('feature_importance.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✓ Feature importance chart saved as 'feature_importance.png'")
else:
    print(f"\n⚠ {best_model_name} does not provide feature importance.")

## Residual Analysis

In [ ]:
# Residual analysis
residuals_test = y_test - y_test_pred

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Residual plot
axes[0].scatter(y_test_pred, residuals_test, alpha=0.5, s=30, color='purple', edgecolors='black', linewidth=0.5)
axes[0].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[0].set_xlabel('Predicted Settle Time (s)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Residuals', fontsize=12, fontweight='bold')
axes[0].set_title(f'Residual Plot - {best_model_name}', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Residual histogram
axes[1].hist(residuals_test, bins=40, color='orange', edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Residuals', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Frequency', fontsize=12, fontweight='bold')
axes[1].set_title(f'Residual Distribution - {best_model_name}', fontsize=14, fontweight='bold')
axes[1].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('residual_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Residual analysis saved as 'residual_analysis.png'")

# Residual statistics
print("\n📈 RESIDUAL STATISTICS")
print("=" * 60)
print(f"  Mean: {residuals_test.mean():.6f}")
print(f"  Std Dev: {residuals_test.std():.6f}")
print(f"  Min: {residuals_test.min():.6f}")
print(f"  Max: {residuals_test.max():.6f}")
print("=" * 60)

## Summary Statistics

In [ ]:
# Create comprehensive summary
print("\n" + "="*80)
print("📊 FINAL SUMMARY")
print("="*80)
print(f"\n✓ Total Simulations Generated: {len(simulation_data)}")
print(f"✓ Input Features: {len(feature_columns)}")
print(f"✓ Output Metrics: {len(output_metrics)}")
print(f"✓ ML Models Compared: {len(results_df)}")
print(f"\n🏆 Best Performing Model: {best_model_name}")
print(f"   - Test R² Score: {best_model_r2:.6f}")
print(f"   - Test RMSE: {best_model_rmse:.6f}")
print(f"   - Performance: {best_model_r2 * 100:.2f}% variance explained")

print(f"\n📈 Top 3 Models:")
for idx in range(min(3, len(results_df))):
    print(f"   {idx+1}. {results_df.loc[idx, 'Model']:20s} - R²: {results_df.loc[idx, 'Test R²']:.6f}")

print("\n" + "="*80)
print("✅ ASSIGNMENT COMPLETED SUCCESSFULLY!")
print("="*80)

## Conclusion

This assignment successfully demonstrated:
1. ✅ Implementation of a physics-based pendulum simulator
2. ✅ Definition of parameter bounds for realistic simulations
3. ✅ Generation of 1000 synthetic simulations
4. ✅ Comparison of 10 different machine learning models
5. ✅ Identification of the best performing model
6. ✅ Comprehensive visualization and analysis

The pendulum simulator provides a rich dataset for machine learning, demonstrating how simulation tools can generate training data for predictive models in physics-based systems.